In [1]:
import spikeinterface.full as si
import torch
import gc
import os
from pathlib import Path
import shutil
import pandas as pd
import time

def run_kilosort_pipeline(basefolders, n_jobs=-4):
    """
    Run Kilosort4 and analysis pipeline for each basefolder in basefolders.

    Parameters
    ----------
    basefolders : list[str or Path]
        List of folders containing raw SpikeGLX data.
    n_jobs : int
        Number of jobs for parallel processing (default: -3)
    """
    global_job_kwargs = dict(n_jobs=n_jobs, chunk_duration="5s", progress_bar=True)
    si.set_global_job_kwargs(**global_job_kwargs)

    for basefolder in basefolders:
        basefolder = Path(basefolder)
        metapath = basefolder / 'Meta'
        metapath.mkdir(exist_ok=True, parents=True)

        print(f"\n=== Processing {basefolder} ===")

        try:
            # --- Load data
            recording = si.read_spikeglx(basefolder, stream_id='imec0.ap', load_sync_channel=False)
            lfp = si.read_spikeglx(basefolder, stream_id='imec0.lf', load_sync_channel=False)
            event = si.read_spikeglx(basefolder, stream_id='nidq', load_sync_channel=False)
            print(recording)

            # --- Preprocessing
            rec1 = si.highpass_filter(recording, freq_min=400.)
            rec1 = si.phase_shift(rec1)
            bad_channel_ids, channel_labels = si.detect_bad_channels(rec1, method='coherence+psd')
            print(f"Bad channels: {bad_channel_ids}")
            rec1 = si.interpolate_bad_channels(rec1, bad_channel_ids=bad_channel_ids)
            rec1 = si.common_reference(rec1, operator="median", reference="global")

            # --- Sorting
            sorted_folder = basefolder / 'sorted'
          
            Sorting_KS4 = si.read_sorter_folder(sorted_folder)

            # --- Analyzer setup
            analyzer = si.create_sorting_analyzer(Sorting_KS4, rec1, sparse=True, format="memory")

            analyzer.compute(['random_spikes', 'waveforms', 'templates', 'noise_levels',
                              'unit_locations', 'correlograms'], **global_job_kwargs)
            analyzer.compute('spike_amplitudes')
            analyzer.compute('principal_components', n_components=5,
                             mode="by_channel_local", **global_job_kwargs)

            # --- Metrics
            metric_names = ['firing_rate', 'presence_ratio', 'snr',
                            'isi_violation', 'amplitude_cutoff', 'amplitude_median']
            metrics = si.compute_quality_metrics(analyzer, metric_names=metric_names)

            # --- Filtering units
            amplitude_cutoff_thresh = 0.1
            isi_violations_ratio_thresh = 0.5
            presence_ratio_thresh = 0.9

            query = f"(amplitude_cutoff < {amplitude_cutoff_thresh}) & " \
                    f"(isi_violations_ratio < {isi_violations_ratio_thresh}) & " \
                    f"(presence_ratio > {presence_ratio_thresh})"

            keep_units = metrics.query(query)
            keep_unit_ids = keep_units.index.values
            analyzer_clean = analyzer.select_units(
                keep_unit_ids,
                folder=basefolder / 'analyzer_clean',
                format='binary_folder'
            )
            print(analyzer)
            print(analyzer_clean)

            # --- Export to Phy
            phy_folder = basefolder / 'sorted' / 'phy'
            si.export_to_phy(analyzer_clean, output_folder=phy_folder, **global_job_kwargs)

            print(f"✅ Finished {basefolder}")
        
        except Exception as e:
            print(f"❌ Error processing {basefolder}: {e}")
        
        finally:
            # --- Memory cleanup
            del recording, lfp, event, rec1
            if 'Sorting_KS4' in locals(): del Sorting_KS4
            if 'analyzer' in locals(): del analyzer
            if 'analyzer_clean' in locals(): del analyzer_clean
            gc.collect()

            
            torch.cuda.empty_cache()
            time.sleep(3)
            print(f"🧹 Memory cleared for {basefolder}")

# Example usage
basefolders = [
    r"D:\Data\raw\7644_recall",  
]
run_kilosort_pipeline(basefolders)



=== Processing D:\Data\raw\7644_recall ===
SpikeGLXRecordingExtractor: 384 channels - 29999.900000 Hz - 1 segments - 139,385,537 samples 
                            4,646.20s (1.29 hours) - int16 dtype - 99.70 GiB
Bad channels: ['imec0.ap#AP191']


estimate_sparsity (workers: 13 processes):   0%|          | 0/930 [00:00<?, ?it/s]

compute_waveforms (workers: 13 processes):   0%|          | 0/930 [00:00<?, ?it/s]

noise_level (workers: 13 processes):   0%|          | 0/20 [00:00<?, ?it/s]

spike_amplitudes (workers: 13 processes):   0%|          | 0/930 [00:00<?, ?it/s]

...

C:\Users\Freitag\AppData\Local\anaconda3\envs\si_env\lib\site-packages\spikeinterface\qualitymetrics\misc_metrics.py:1066: UserWarning: Some units have too few spikes : amplitude_cutoff is set to NaN
  warnings.warn(f"Some units have too few spikes : amplitude_cutoff is set to NaN")


SortingAnalyzer: 384 channels - 309 units - 1 segments - memory - sparse - has recording
Loaded 9 extensions: random_spikes, waveforms, templates, noise_levels, unit_locations, correlograms, spike_amplitudes, principal_components, quality_metrics
SortingAnalyzer: 384 channels - 55 units - 1 segments - binary_folder - sparse - has recording
Loaded 9 extensions: random_spikes, waveforms, templates, noise_levels, unit_locations, correlograms, spike_amplitudes, principal_components, quality_metrics


write_binary_recording (workers: 13 processes):   0%|          | 0/930 [00:00<?, ?it/s]

extract PCs (workers: 13 processes):   0%|          | 0/930 [00:00<?, ?it/s]

Run:
phy template-gui  D:\Data\raw\7644_recall\sorted\phy\params.py
✅ Finished D:\Data\raw\7644_recall
🧹 Memory cleared for D:\Data\raw\7644_recall
